<a href="https://colab.research.google.com/github/Nanasz/Kelompok7-DataMining2/blob/main/Kelompok_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CASE STUDY 02 MixITup**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

##Load Data Excel

In [ ]:
import warnings; warnings.simplefilter('ignore')

try:
    import google.colab; IN_COLAB = True
    print("Installing the required modules")
    !pip install --upgrade umap-learn
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/tau_unsup.py
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

Installing the required modules
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=71fd26a55926794b8e7c66fa8407d52b0f38e783c928e11932c9f75b6766f770
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=5afa90ae8ad4495d994d0d588dc7f3b2605a4dcd3b7a59ec606c8dfc8aeee7ce
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent
--2022-10-12 01:30:12--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/tau_unsup.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199

In [ ]:
# Running Locally, yakinkan "file_" berada di folder "data"
try :
    data = pd.read_excel("data/Data_Studi_Kasus_2.xlsx", sheet_name=0)
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/Nanasz/Kelompok7-DataMining2/main/Data_Studi_Kasus_2.xlsx
    data = pd.read_excel("data/Data_Studi_Kasus_2.xlsx", sheet_name=0)
data.shape

--2022-10-12 01:30:13--  https://raw.githubusercontent.com/Nanasz/Kelompok7-DataMining2/main/Data_Studi_Kasus_2.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22293 (22K) [application/octet-stream]
Saving to: ‘data/Data_Studi_Kasus_2.xlsx’

Data_Studi_Kasus_2. 100%[===================>]  21.77K  --.-KB/s    in 0.001s  

2022-10-12 01:30:13 (30.8 MB/s) - ‘data/Data_Studi_Kasus_2.xlsx’ saved [22293/22293]



(200, 10)

In [ ]:
data

,ID,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,101,Wanita,49,13114754,52,1,Jakarta,1.0,Sabtu,2
1,102,Wanita,32,33606557,86,1,Depok,1.0,Minggu,6
2,103,Pria,51,15983606,41,1,Jakarta,2.0,Minggu,3
3,104,Pria,19,12704918,55,0,Jakarta,1.0,Rabu,7
4,105,Wanita,38,20081967,42,1,Jakarta,2.0,Senin,8
...,...,...,...,...,...,...,...,...,...,...
195,296,Pria,31,4098360,73,1,Depok,1.0,Senin,0
196,297,Pria,57,15983606,51,1,Jakarta,1.0,Minggu,5
197,298,Wanita,35,43032786,79,1,Depok,1.0,Senin,7
198,299,Wanita,23,1229508,94,1,Depok,0.0,Selasa,0


In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import umap, numpy as np, tau_unsup as tau, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
from sklearn import cluster, datasets
from sklearn.metrics import silhouette_score as siluet
from sklearn.metrics.cluster import homogeneity_score as purity
from sklearn.metrics import normalized_mutual_info_score as NMI 

sns.set(style="ticks", color_codes=True)
random_state = 99

## Mengatasi MV

In [ ]:
data.isna().sum()

ID                       0
Jenis Kelamin            4
Umur                     0
Pendapatan-per-bulan     0
skor-pengeluaran         0
Menikah                 12
Kota                     0
Promo                    6
HariFav                  0
FavFlavor                0
dtype: int64

kita tahu bahwa dalam dataset semua kolom tidak ada nilai yang kosong kecuali kolom Jenis Kelamin dengan 4 missing value, kolom Menikah dengan 12 missing value dan kolom Promo dengan 6 missing value

In [ ]:
# Mengatasi MV
data.fillna(0, inplace = True)

In [ ]:
data

,ID,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,101,Wanita,49,13114754,52,1,Jakarta,1.0,Sabtu,2
1,102,Wanita,32,33606557,86,1,Depok,1.0,Minggu,6
2,103,Pria,51,15983606,41,1,Jakarta,2.0,Minggu,3
3,104,Pria,19,12704918,55,0,Jakarta,1.0,Rabu,7
4,105,Wanita,38,20081967,42,1,Jakarta,2.0,Senin,8
...,...,...,...,...,...,...,...,...,...,...
195,296,Pria,31,4098360,73,1,Depok,1.0,Senin,0
196,297,Pria,57,15983606,51,1,Jakarta,1.0,Minggu,5
197,298,Wanita,35,43032786,79,1,Depok,1.0,Senin,7
198,299,Wanita,23,1229508,94,1,Depok,0.0,Selasa,0


Mengganti semua nilai yang kosong dengan angka nol di pandas dataframe.

## Melihat tipe data dalam tabel Mixltup

In [ ]:
# Tipe Datanya : DataFrame (df)
print(type(data))
# Ukuran Data
N, P = data.shape
'baris = ', N, ', Kolom = ', P

<class 'pandas.core.frame.DataFrame'>


('baris = ', 200, ', Kolom = ', 10)

Tipe data yang dimiliki adalah pa

In [ ]:
data.dtypes

ID                        int64
Jenis Kelamin            object
Umur                     object
Pendapatan-per-bulan      int64
skor-pengeluaran          int64
Menikah                  object
Kota                     object
Promo                   float64
HariFav                  object
FavFlavor                 int64
dtype: object

## Mengubah tipe data

In [ ]:
data['Jenis Kelamin'] = data['Jenis Kelamin'].astype('category')
data['Pendapatan-per-bulan'] = data['Pendapatan-per bulan'].astype('float64')
data['Jenis Kelamin'] = data['Jenis Kelamin'].astype('category')
data['Umur'] = data['Umur'].astype('int64')

KeyError: ignored

In [ ]:
# menghapus data yang tidak berguna
data.drop("No", axis=1, inplace=True)

In [ ]:
# mengganti tipe data
data.types

In [ ]:
# memeriksa duplikasi data
print(data.shape)
data.duplicated().sum()

In [ ]:
#menghapus entri yang memiliki data duplikat
data.drop_duplicates(inplace=True)
print(data.duplicated().sum()) # tidak ada lagi duplikat
print(data.shape) # mengecek kembali dengan print data.shape


In [ ]:
# Memilih hanya variable dengan tipe tertentu
data_num = data.select_dtypes(include = ['float64', 'int64'])
data_num.head()
# Perhatikan data_num adalah variable df baru! ... (hati-hati di data yang besar)


In [ ]:
# Memilih hanya variable dengan tipe tertentu
data_cat = data.select_dtypes(include = ['category', 'datetime64', 'object'])
data_cat.head()

In [ ]:
df = pd.get_dummies(data['Kamar Tidur Pembantu'], prefix='rumah')
df.head()

In [ ]:
df2 = pd.concat([data, df], axis = 1)
df2.head().transpose()

In [ ]:
# Memilih beberapa kolom secara manual
X = data[['Harga_Juta','Luas Tanah','Luas Bangunan']] 
X[11:20]

In [ ]:
# cek missing value
gdcu.isnull().sum()

sns.heatmap(data2.isnull(), cbar=False)
plt.title('Heatmap Missing Value')
plt.show()

In [ ]:
# menghapus missing value
gdcu.drop('floor_count', axis = 1,inplace = True)
gdcu.isnull().sum()

In [ ]:
# outlier

#outlier pada kolom wind_speed
# Deteksi Outlier menggunakan nilai IQR

Q1 = gdcu.wind_speed.quantile(0.25)
Q3 = gdcu.wind_speed.quantile(0.75)
IQR = Q3 - Q1
print('Q1 =', Q1)
print('Q2 =', Q3)
print('IQR =', IQR)

In [ ]:
# Menentukan batas bawah dan batas atas

batasBawah = Q1 - (1.5 * IQR)
batasAtas = Q3 + (1.5 * IQR)
print('batas bawah =', batasBawah)
print('batas atas =', batasAtas)

In [ ]:
#Mengecek outlier menggunakan visualisasi boxplot
fig, ax = plt.subplots(figsize = (15,4))
sns.boxplot(x = gdcu.wind_speed).set_title('Wind Speed Boxplot')

In [ ]:
# Deteksi Outlier menggunakan nilai IQR
Q1 = gdcu.wind_speed.quantile(0.25)
Q3 = gdcu.wind_speed.quantile(0.75)
IQR = Q3 - Q1
print('Q1 =', Q1)
print('Q2 =', Q3)
print('IQR =', IQR)

In [ ]:
# Menentukan batas bawah dan batas atas

batasBawah = Q1 - (1.5 * IQR)
batasAtas = Q3 + (1.5 * IQR)
print('batas bawah =', batasBawah)
print('batas atas =', batasAtas)

In [ ]:
# handle outlier dengan replace menggantinya dengan nilai mean

gdcu['wind_speed'] = gdcu.wind_speed.mask(gdcu.wind_speed > batasAtas, gdcu.wind_speed.mean())
gdcu['wind_speed'] = gdcu.wind_speed.mask(gdcu.wind_speed < batasBawah, gdcu.wind_speed.mean())
fig, ax = plt.subplots(figsize = (15,7))
fig = sns.boxplot(x = gdcu.wind_speed).set_title('Wind Speed Boxplot')

In [ ]:
data.info()

In [ ]:
# menghapus kolom tidak penting
data.drop(columns=['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'], axis = 1, inplace = True)

In [ ]:
data.dtypes

In [ ]:
## Mengubah tipe data yang diperlukan
data['LT'] = data['LT'].astype('float64')

In [ ]:
print(data.shape)
data.duplicated().sum()